In [1]:
import pandas as pd
from trackml.dataset import load_event
from trackml.dataset import load_dataset
from trackml.randomize import shuffle_hits
from trackml.score import score_event 
%run utils.ipynb
import time

In [ ]:
df = pd.read_csv('sample_submission.csv')

In [ ]:
df.head(10)

In [ ]:
%ls data/test_data

In [ ]:
itr =  load_dataset('data/test_data', parts=['hits','cells'])
    

In [ ]:
event_id, hits, cells = next(itr)

In [ ]:
hits.head()

In [2]:
def remove_disk_hits(hits_df):
    hits_df['is_cylinder'] = hits_df.apply(lambda x: is_cylinder(x['volume_id'], x['layer_id']), axis = 1)
    
    return hits_df[hits_df.is_cylinder == True], hits_df[hits_df.is_cylinder == False]


"""
inputs

track_candidates: tensor of shape [num_seeds x 10 x 3] 3 for r,phi,z

track_candidates_map: tensor of shape [num_seeds x 10 x 2] 2 for mse, hit_ids

"""

def prune(track_candidates, track_candidates_map, hits_df, event_id):
    start = time.time()
    cyl_only_hits, non_cyl_hits = remove_disk_hits(hits_df)
    
    bad_hit_ids = non_cyl_hits['hit_id'].values
    dummy_track_id = np.zeros(bad_hit_ids.shape, dtype=np.int32)
    event_ids = np.full(bad_hit_ids.shape, event_id, dtype=np.int32)
    
    ignored_tracks = pd.DataFrame({'event_id':event_ids, 'hit_id':bad_hit_ids, 'track_id':dummy_track_id})
    
    cyl_only_hits['assigned'] = False
    cyl_only_hits['track_id'] = np.nan
    track_id = 1
    
    out = np.take(track_candidates_map,[0], axis=2) # grab mse per hit per track
    track_mses = np.sum(out,axis=1) #calc mse per track
    sorted_idxs=np.argsort(track_mses, axis = 0)
    #flat_idxs = sorted_idxs.flatten()
    sorted_tracks = track_candidates[sorted_idxs]
    sorted_track_mapping = track_candidates_map[sorted_idxs]

    for batchIdx, batch in enumerate(sorted_tracks):
        for trackIdx, track in enumerate(batch):
            temp_hit_ids = []
            valid_track = True

            for hitIdx, hit in enumerate(track): 
                #print(hit)
                hit_id = sorted_track_mapping[batchIdx][trackIdx][hitIdx][1]
                temp_hit_ids.append(hit_id)

            for hit_id in temp_hit_ids:
                if cyl_only_hits.loc[cyl_only_hits['hit_id']==hit_id]['assigned'] is True:
                    valid_track = False
                    break

            if valid_track:
                for hit_id in temp_hit_ids:
                    cyl_only_hits.loc[cyl_only_hits['hit_id']==hit_id, 'assigned'] = True
                    
                    
                    #idx = cyl_only_hits.loc[cyl_only_hits['hit_id']==hit_id].index
                    #print('idx is: ',idx)
                    cyl_only_hits.loc[cyl_only_hits['hit_id']==hit_id,'track_id'] = track_id
                    #cyl_only_hits.at[idx,'assigned'] = track_id
                    #df.at['C', 'x'] = 10
                    
                track_id +=1
    
    dropped_df = cyl_only_hits.drop(['x','y','z','volume_id','layer_id','module_id','is_cylinder','module_id'],axis=1)
    
   
    final_results = dropped_df.fillna(0, downcast={np.float32,np.int32})     
        
    
    
    
    print('elasped time: '+str(time.time()-start))
    return ignored_tracks, final_results

    
    


In [3]:
mapping = np.load('./mse_hitid.npy')
track_cand = np.load('./hits_from_tracks.npy')

In [5]:
m_slice = mapping[:10]
track_slice = track_cand[:10]

In [7]:
m_slice

array([[[0.00000000e+00, 1.63080000e+04],
        [0.00000000e+00, 2.42910000e+04],
        [0.00000000e+00, 3.06020000e+04],
        [1.17194073e+03, 3.66910000e+04],
        [8.91501026e+02, 6.62900000e+04],
        [5.44395213e+01, 7.24370000e+04],
        [8.22152534e+02, 7.84130000e+04],
        [1.60521265e+04, 6.42130000e+04],
        [7.04548314e+03, 9.85500000e+04],
        [9.89070871e+03, 9.81160000e+04]],

       [[0.00000000e+00, 1.63440000e+04],
        [0.00000000e+00, 2.43020000e+04],
        [0.00000000e+00, 3.06050000e+04],
        [6.90764819e+02, 3.66910000e+04],
        [1.33385688e+03, 6.62940000e+04],
        [3.16603324e+02, 7.24430000e+04],
        [3.06423993e+02, 7.84180000e+04],
        [1.72228900e+04, 6.29950000e+04],
        [2.81504912e+03, 9.81440000e+04],
        [9.33109259e+03, 9.77370000e+04]],

       [[0.00000000e+00, 1.63900000e+04],
        [0.00000000e+00, 2.43450000e+04],
        [0.00000000e+00, 3.06340000e+04],
        [1.23671562e+02, 3.673

In [ ]:
m_slize[:]

In [ ]:
((9/10)*16517)/(3600)

In [ ]:
track_slice.shape

In [ ]:
mapping = np.load('./mse_hitid.npy')
track_cand = np.load('./hits_from_tracks.npy')
hits, _, _,_ = load_data_single_event(1050)
 
start = time.time()
bad_results, results = prune(track_slice, m_slice, hits, 1050)
print('total time: ', time.time()-start)

In [ ]:
"""


out = np.take(track_mapping,[0], axis=2) # grab mse per hit per track
track_mses = np.sum(out,axis=1) #calc mse per track

sorted_idxs=np.argsort(track_mses, axis = 0)
#flat_idxs = sorted_idxs.flatten()
sorted_tracks = track_candidates[sorted_idxs]

sorted_track_mapping = track_mapping[sorted_idxs]



hits_ref = hits #remember to pass the hits_df in!
hits_ref['assigned']=False
hits_ref['track_id'] = np.nan
track_id = 1
for batchIdx, batch in enumerate(sorted_tracks):
    for trackIdx, track in enumerate(batch):
        temp_hit_ids = []
        valid_track = True
        
        for hitIdx, hit in enumerate(track): 
            #print(hit)
            hit_id = sorted_track_mapping[batchIdx][trackIdx][hitIdx][1]
            temp_hit_ids.append(hit_id)
        
        for hit_id in temp_hit_ids:
            if hits_ref.loc[hits_ref['hit_id']==hit_id]['assigned'].item() is True:
                valid_track = False
                break
        
        
        if valid_track:
            for hit_id in temp_hit_ids:
                hits_ref.loc[hits_ref['hit_id']==hit_id]['assigned'] = True
                hits_ref.loc[hits_ref['hit_id']==hit_id]['track_id'] = track_id
            track_id +=1
            

            
"""             
            
            